## Примеры промпт-инжиниринга с YandexGPT

Для работы с YandexGPT будем использовать фреймворк LangChain. Создадам объект `GPT` для взаимодействия с YandexGPT. 

> Чтобы код ниже работал, нам необходимо поместить файл `config.json` с ключами `folder_id` и `api_key`

Мы будем использовать параметр `model_name='yandexgpt'` для работы с моделью YandexGPT Pro.

In [14]:
from langchain_community.llms import YandexGPT
import json

config = json.load(open('config.json'))
GPT = YandexGPT(api_key = config['api_key'], 
                folder_id = config['folder_id'],
                model_name = 'yandexgpt',
                temperature=1)

print(GPT.invoke('Расскажи анекдот про JSON'))

Клиент заказал сайт, разработчик выдаёт ему счёт. Клиент в недоумении:

— Здесь в два раза больше строк кода, чем на моём старом сайте!

Разработчик:

— Не переживайте, просто новый сайт написан на JSON, а старый — на HTML.


## Извлечение структурированных данных из текста

Предположим, у нас есть отзывы о ресторанах следующего вида:

In [15]:
reviews = ["""
Я посетил ресторан Бургер Кинг летом прошлого года, и был разочарован!
Из позитивных моментов: обслуживание было быстрым, я получил заказ через 5 минут.
Но при этом весь персонал был мрачным, и еда оказалась не очень вкусной. Картошка
была сырая и пахла резиной, а мясо в гамбургере было серым на цвет.
""","""
Я слышал, что в Макдональдсе котлеты готовят не из мяса, и 
сегодня я в этом убедился сам! В котлете попалось что-то жесткое,
и я чуть не сломал зуб!
""","""
Я был а Макдональдсе четыре раза, и каждый раз это было удивительно!
Столько вкусов мороженого я никогда не пробовал! И все официантки за
кассой очень молодые и симпатичные!
""","""
Макдональдс - это прекрасное место, где можно поесть американскую еду:
гамбургеры, картошку фри и конечно же прекрасное мороженое!
Я обычно заказываю биг мак, в котором много вкусного зелёного салата.
Это делает еду полезной и здоровой, что очень хорошо! Спасибо всем официантам,
которые всегда улыбаются и радуются мне!
""","""
Мне нравятся официантки во "Вкусно и точка", они всегда вовремя приносят блюда.
Но при этом вкус блюд не очень.
"""]

Попробуем извлчечь из этих отзывов стуктурированную информацию в формате JSON:

In [16]:
template = '''
Ниже в тройных обратных кавычках приводится отзыв посетителя о ресторане. Пожалуйста,
прочитай этот отзыв, и извлеки из него следующую информацию:
1. Название ресторана (name)
2. Качество обслуживания (service): дробное число от -1.0 (плохо) до 1.0 (хорошо)
3. Качество еды (food): дробное число от -1.0 (плохо) до 1.0 (хорошо)
4. Общая тональность отзыва (tone): дробное число от -1.0 (отрицательный) до 1.0 (положительный)
5. Краткое содержание отзыва (summary) в 10-15 словах
Результат верни в формате JSON такого вида:
{{
  "name" : "...",
  "service" : ...,
  "food" : ...,
  "tone" : ...,
  "summary" : "..."
}}
Верни только JSON, не пиши никаких дополнительных слов.
Вот сам отзыв:
```{}```
'''

print(GPT(template.format(reviews[0])))

{
  "name": "Бургер Кинг",
  "service": 0.5,
  "food": -0.75,
  "tone": -0.6,
  "summary": "Разочарование в обслуживании и качестве еды."
}


Соберём результаты обработки всех отзывов в список, и на основе этого построим таблицу:

In [17]:
import pandas as pd

res = []
for x in reviews:
  r = GPT(template.format(x))
  r = r[r.find('{'):]
  r = r[:r.find('}')+1]
  print(r)
  res.append(json.loads(r))

df = pd.DataFrame(res)
df

{
  "name": "Бургер Кинг",
  "service": 0.5,
  "food": -0.75,
  "tone": -0.5,
  "summary": "Посетитель разочарован: быстрое обслуживание, но мрачный персонал и невкусная еда."
}
{
  "name": "Макдональдс",
  "service": -1.0,
  "food": -1.0,
  "tone": -1.0,
  "summary": "В котлете попалось что-то жёсткое."
}
{
  "name": "Макдональдс",
  "service": 1.0,
  "food": 1.0,
  "tone": 1.0,
  "summary": "Много вкусов мороженого. Симпатичные официантки."
}
{
  "name": "Макдональдс",
  "service": 1.0,
  "food": 1.0,
  "tone": 1.0,
  "summary": "Полезный и вкусный бургер с салатом."
}


Retrying langchain_community.llms.yandex.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised _MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.RESOURCE_EXHAUSTED
	details = "ai.textGenerationCompletionSessionsCount.count gauge quota limit exceed: allowed 1 requests"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:158.160.54.160:443 {created_time:"2024-07-08T21:12:53.9533124+00:00", grpc_status:8, grpc_message:"ai.textGenerationCompletionSessionsCount.count gauge quota limit exceed: allowed 1 requests"}"
>.


{
  "name": "Вкусно и точка",
  "service": 1.0,
  "food": -0.5,
  "tone": 0.5,
  "summary": "Официантки во «Вкусно и точка» всегда вовремя приносят блюда."
}


,name,service,food,tone,summary
0,Бургер Кинг,0.5,-0.75,-0.5,"Посетитель разочарован: быстрое обслуживание, ..."
1,Макдональдс,-1.0,-1.00,-1.0,В котлете попалось что-то жёсткое.
2,Макдональдс,1.0,1.00,1.0,Много вкусов мороженого. Симпатичные официантки.
3,Макдональдс,1.0,1.00,1.0,Полезный и вкусный бургер с салатом.
4,Вкусно и точка,1.0,-0.50,0.5,Официантки во «Вкусно и точка» всегда вовремя ...


Теперь легко можем посчитать средние значения показателей по всем ресторанам:

In [18]:
df[['service','food','tone','name']].groupby('name').mean()

,service,food,tone
name,,,
Бургер Кинг,0.500000,-0.750000,-0.500000
Вкусно и точка,1.000000,-0.500000,0.500000
Макдональдс,0.333333,0.333333,0.333333


## Извлечение рисунков из текста

Интересный приём - извлечение структурных диаграмм из текста. YandexGPT не может рисовать, но он может генерировать текстовые описания диаграмм и mindmap-ов в таких форматах, как [mermaid.js](https://mermaid.js.org), [PlantUML](https://plantuml.com) или [Markmap.js](http://markmap.js.org).

Рассмотрим, как извлечь блок-схему из описания простого рецепта:

In [84]:
recipe = """
Ниже приводится рецепт моего любимого блюда. Возмём клубнику и положим в воду. Одновременно с этим положим
сметану в небольшую мисочку и добавим ванилина. Когда клубника отмокнет, нарежем её небольшими ломтиками и положим в 
сметану. Сверху присыпем небольшим количеством корицы. Блюдо готово!
""" 

template = """
Ниже в тройных обратных кавычках приводится рецепт блюда. Пожалуйста, прочитай его, и представь
шаги по его приготовлению в виде диаграммы с прямоугольниками и стрелочками в формате mermaid.js.
Верни только текстовое описание диаграммы, ничего лишнего. Для шагов диаграммы придумай
короткие описания в 1-2 слова.
Рецепт: ```{}```
"""

GPT.temperature = 0.1
res = GPT(template.format(recipe))
print(res)

Конечно, вот текстовое описание диаграммы по шагам приготовления блюда:

```mermaid
graph LR
A[Возьмём клубнику] --> B[Положим в воду]
C[Положим сметану в мисочку] --> D[Добавим ванилина]
B --> E[Нарежем клубнику ломтиками]
E --> F[Поместим клубнику в сметану]
F --> G[Присыпем корицей]
G --> H[Блюдо готово!]
```


Чтобы показать картинку, используем сервис для рендеринга Mermaid.js под названием Mermaid.ink:

In [107]:
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt
import zlib 

def compress(data):
    c = zlib.compressobj(9, zlib.DEFLATED, 15, 8,zlib.Z_DEFAULT_STRATEGY)
    compressed_data = c.compress(data)
    compressed_data += c.flush()
    return compressed_data

def render_mermaid(graph):
    jGraph = { "code": graph, "mermaid": {"theme": "default"}}
    byteStr = json.dumps(jGraph).encode('utf-8')
    deflated = compress(byteStr)
    base64_bytes = base64.b64encode(deflated)
    base64_string = base64_bytes.decode("ascii").replace('/','_')
    display(Image(url="https://mermaid.ink/img/pako:" + base64_string))

x = res[res.find('```'):].replace('```mermaid','')
x = x[:x.find('```')]

render_mermaid(x)